# Reporting on sales data

## 📖 Background
The company sells motorcycle parts, and operates three warehouses in a large metropolitan area.

### Objective
Analyze past sales data.
* Capture sales by payment method.
* Find average unit price for each product line.
* Crate visualizations 
* Include other relevant findings


## 💾 The data

#### The sales data has the following fields:
- "date" - The date, from June to August 2021.
- "warehouse" - The company operates three warehouses: North, Central, and West.
- "client_type" - There are two types of customers: Retail and Wholesale.
- "product_line" - Type of products purchased.
- "quantity" - How many items were purchased.
- "unit_price" - Price per item sold.
- "total" - Total sale = quantity * unit_price.
- "payment" - How the client paid: Cash, Credit card, Transfer.

In [ ]:
# Importing the pandas module
import pandas as pd

# Reading in the sales data
df = pd.read_csv('data/sales_data.csv', parse_dates=['date'])

# Take a look at the first datapoints
df.head()

,date,warehouse,client_type,product_line,quantity,unit_price,total,payment
0,2021-06-01,Central,Retail,Miscellaneous,8,16.85,134.83,Credit card
1,2021-06-01,North,Retail,Breaking system,9,19.29,173.61,Cash
2,2021-06-01,North,Retail,Suspension & traction,8,32.93,263.45,Credit card
3,2021-06-01,North,Wholesale,Frame & body,16,37.84,605.44,Transfer
4,2021-06-01,Central,Retail,Engine,2,60.48,120.96,Credit card


In [309]:
df.info() # firt check of nulls and datatypes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          1000 non-null   datetime64[ns]
 1   warehouse     1000 non-null   object        
 2   client_type   1000 non-null   object        
 3   product_line  1000 non-null   object        
 4   quantity      1000 non-null   int64         
 5   unit_price    1000 non-null   float64       
 6   total         1000 non-null   float64       
 7   payment       1000 non-null   object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 62.6+ KB


In [308]:
df.describe(include='all', datetime_is_numeric=True)

,date,warehouse,client_type,product_line,quantity,unit_price,total,payment
count,1000,1000,1000,1000,1000.000000,1000.000000,1000.000000,1000
unique,NaN,3,2,6,NaN,NaN,NaN,3
top,NaN,Central,Retail,Breaking system,NaN,NaN,NaN,Credit card
freq,NaN,480,775,230,NaN,NaN,NaN,659
mean,2021-07-15 00:05:45.600000,NaN,NaN,NaN,9.395000,30.322040,289.113000,NaN
min,2021-06-01 00:00:00,NaN,NaN,NaN,1.000000,10.030000,10.350000,NaN
25%,2021-06-24 00:00:00,NaN,NaN,NaN,4.000000,21.085000,93.687500,NaN
50%,2021-07-14 00:00:00,NaN,NaN,NaN,6.500000,28.570000,178.360000,NaN
75%,2021-08-06 00:00:00,NaN,NaN,NaN,10.000000,37.917500,321.690000,NaN
max,2021-08-28 00:00:00,NaN,NaN,NaN,40.000000,66.620000,2546.330000,NaN


In [ ]:
print("Fecha inicio: ",df.date.min(),'\nFecha Fin: ',df.date.max())

Fecha inicio:  2021-06-01 00:00:00 
Fecha Fin:  2021-08-28 00:00:00


# Total sales for each payment method

The payment method in which the highest sales are made is through transfers (55%) but in number of sales it only represents 22.5%, while with cash there are fewer sales both in quantity and amount. It is with a credit card that the greatest number of sales occurs, almost 66%.

In [396]:
total_sales =   np.round(df.groupby(['payment'])[['total']].
                agg(total_sales = ('total', 'sum'),
                    q_sales = ('total', 'count')).
                reset_index(),0)
total_sales['percentage_total_sales'] = np.round(total_sales['total_sales'] / total_sales['total_sales'].sum()*100,1)
total_sales['perc_q_sales'] = np.round(total_sales['q_sales'] / total_sales['q_sales'].sum()*100,1)
total_sales

,payment,total_sales,q_sales,percentage_total_sales,perc_q_sales
0,Cash,19199.0,116,6.6,11.6
1,Credit card,110272.0,659,38.1,65.9
2,Transfer,159642.0,225,55.2,22.5


In [397]:
import plotly.express as px

fig = px.bar(total_sales,
            x='payment',
            y="total_sales",
            color='payment',
            title="Total Sales for Payment Method",
            hover_data=['percentage_total_sales','perc_q_sales','q_sales'],
            text_auto=True)
fig.update_yaxes(title_text='Total Sales')
fig.update_xaxes(title_text='Payment Method')
fig.update(layout_showlegend=False)
fig.update_layout(autosize=False,
                    width=800,
                    height=400)
fig.show()


## Average purchase by Payment method


In [422]:
# average purchase by Payment method
avg_sales_payment =     np.round((df.groupby(['payment'])[['total']].
                        agg(avg_sale = ('total', 'mean')).
                        reset_index()),2)
avg_sales_payment

,payment,avg_sale
0,Cash,165.51
1,Credit card,167.33
2,Transfer,709.52


# Average unit price for each product line, and units sold

The product line with the highest price per unit is the engines, while the breaking system is the lowest. 

In [436]:
# What is the average unit price for each product line?
average_unite_price = np.round(df.groupby(['product_line'])[['quantity','unit_price']].
                        agg(average_price =('unit_price','mean'), units_solds =('quantity','sum')).
                        sort_values(by='average_price',ascending=False).
                        reset_index(),2)
average_unite_price

,product_line,average_price,units_solds
0,Engine,60.09,627
1,Frame & body,42.83,1619
2,Suspension & traction,33.97,2145
3,Electrical system,25.59,1698
4,Miscellaneous,22.81,1176
5,Breaking system,17.74,2130


In [446]:
fig = px.bar(average_unite_price,
            y='product_line',
            x="units_solds",
            color='product_line',
            title="Units Sold",
            text_auto=True,
            orientation='h')
fig.update_xaxes(title_text='Units Sold')
fig.update_yaxes(title_text='Product Line')
fig.update(layout_showlegend=False)
fig.update_layout(yaxis={'categoryorder':'total ascending'},
    autosize=False,
    width=800,
    height=400)
fig.show()

In [435]:
fig = px.bar(average_unite_price,
            y='product_line',
            x="average_price",
            color='product_line',
            title="Average Unit Price",
            text_auto=True,
            orientation='h')
fig.update_xaxes(title_text='Avg Unit Price')
fig.update_yaxes(title_text='Product Line')
fig.update(layout_showlegend=False)
fig.update_layout(
    autosize=False,
    width=800,
    height=400,)
fig.show()

## Average of units price by client type/warehouse

It can be seen that in general there is not much difference between retail and wholesale prices, and in fact in some cases retail prices are slightly lower, this curiosity, for example, occurs in the warehouses located in the west/central area. Also, in the west area the only sales of engines were at retail.
The distribution of prices for all product lines and types of customers share similarities. It can be seen that the price per unit, of the engines product line, has less dispersion when it is wholesale.


In [407]:
average_unite_price_w = np.round(pd.pivot_table(df,values='unit_price',
                                                index='product_line',
                                                columns=['warehouse','client_type'],
                                                aggfunc=np.mean),1)
average_unite_price_w.style\
      .format('{:.1f}')\
      .highlight_max(color = 'green',axis=1)\
      .highlight_min(color = 'green',axis=1)\
      .highlight_null(null_color='black')

In [386]:
fig = px.box(df, x='product_line', y='unit_price',color='client_type')
fig.update_layout(
    autosize=False,
    width=800,
    height=400,)
fig.show()

# Purchase by Product Line (Avg & Total Sales, $ and Q)

The products with the largest units sold are two, the braking and suspension & traction system, the latter being the one that represents the largest amount in the company's total sales (25.3%).
The motor product line, surely due to its high cost, has fewer units sold.

Central Warehouse is the one that sells the most units in all product lines.
On only two products, "Frame & Body" and "Suspension & Traction," North Warehouse comes close to their numbers. West warehouse being the one with the lowest sales.

In [427]:
units_solds = np.round(df.groupby(['product_line'])[['quantity']].
                        agg(total_units_sold =('quantity', 'sum')).
                        sort_values(by=['total_units_sold'], ascending=False).
                        reset_index(),2)
units_solds

,product_line,total_units_sold
0,Suspension & traction,2145
1,Breaking system,2130
2,Electrical system,1698
3,Frame & body,1619
4,Miscellaneous,1176
5,Engine,627


In [417]:
# average purchase by Product Line
avg_sales_product =     np.round(df.groupby(['product_line','warehouse'])[['total','quantity']].
                        agg(avg_sale = ('total', 'mean'),
                            avg_q_prod_sale = ('quantity', 'mean'),
                            median_q_prod_sale=('quantity', 'median'),
                            total_units_sold =('quantity', 'sum')).
                        sort_values(by=['product_line','avg_sale'], ascending=False).
                        reset_index(),2)
avg_sales_product


,product_line,warehouse,avg_sale,avg_q_prod_sale,median_q_prod_sale,total_units_sold
0,Suspension & traction,North,334.61,9.62,7.5,866
1,Suspension & traction,Central,311.15,9.33,6.0,980
2,Suspension & traction,West,309.94,9.06,6.0,299
3,Miscellaneous,West,243.32,9.96,6.5,259
4,Miscellaneous,North,226.81,9.97,6.0,379
5,Miscellaneous,Central,210.70,9.28,7.0,538
6,Frame & body,North,481.32,11.44,7.0,675
7,Frame & body,Central,385.14,8.95,6.0,698
8,Frame & body,West,365.05,8.48,7.0,246
9,Engine,Central,757.31,12.47,8.0,449


In [424]:
fig = px.bar(avg_sales_product, 
                x="warehouse",
                y="total_units_sold",
                color="warehouse",
                barmode="group",
                facet_col='product_line',
                hover_data=['avg_q_prod_sale','median_q_prod_sale','avg_sale'])
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
#fig.update_traces(showlegend=False)
fig.update(layout_showlegend=True)
fig.update_layout(
    autosize=False,
    width=800,
    height=400)
fig.update_xaxes(visible=False, showticklabels=True,title_text='Warehouse')

fig.show()

In [414]:
#Total Sales by Product Line
total_sales_product =   (df.groupby(['product_line'])[['total']].
                        agg(total_sales = ('total', 'sum'), q_sales=('total', 'count')).
                        sort_values(by='total_sales', ascending=False).
                        reset_index())
total_sales_product['perc_total_sales']  =  np.round(total_sales_product['total_sales'] /  total_sales_product['total_sales'].sum()*100,2)         
print(total_sales_product)

            product_line  total_sales  q_sales  perc_total_sales
0  Suspension & traction     73014.21      228             25.25
1           Frame & body     69024.73      166             23.87
2      Electrical system     43612.71      193             15.09
3        Breaking system     38350.15      230             13.26
4                 Engine     37945.38       61             13.12
5          Miscellaneous     27165.82      122              9.40


# Average purchase value by Warehouse


Of total sales, 49% comes from the Central Warehouse, while only 16% comes from the West Warehouse

In [233]:
import numpy as np
total_sales_warehouse =     np.round(df.groupby(['warehouse'])[['total']].
                            agg(total_sales = ('total', 'sum'),q_sales=('total', 'count')).
                            reset_index(),1)
total_sales_warehouse['avg_sale'] = np.round(total_sales_warehouse.total_sales/total_sales_warehouse.q_sales,1)
total_sales_warehouse['perc_total_sales'] = np.round((total_sales_warehouse.total_sales/total_sales_warehouse.total_sales.sum())*100,2)

print(total_sales_warehouse)

  warehouse  total_sales  q_sales  avg_sale  perc_total_sales
0   Central     141982.9      480     295.8             49.11
1     North     100203.6      340     294.7             34.66
2      West      46926.5      180     260.7             16.23


In [234]:

fig = px.bar(total_sales_warehouse,
            x='warehouse',
            y="total_sales",
            color='warehouse',
            title='Total Sales by Warehouse',hover_data=['q_sales','avg_sale','perc_total_sales'],
            text_auto=True)
fig.update_yaxes(title_text='Total Sales')
fig.update_xaxes(title_text='Warehouse')
fig.update_layout(
    autosize=False,
    width=800,
    height=400)
fig.show()